# Exercices DataViz

In [1]:
import numpy as np # algèbre linéaire
import pandas as pd # procès de données, CSV file I/O (e.g. pd.read_csv)

# plotly
# import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# librairie word cloud
from wordcloud import WordCloud

# librairie matplotlib
import matplotlib.pyplot as plt

# Permet d'afficher les données disponibles dans le répertoire data
import os
print(os.listdir(".")) 
# Chargement des données qui seront utilisées.
timeData = pd.read_csv("timesData.csv")
timeData.head(100)
timeData.info()

['Tutorial_plotly_DataDev_Simplon_L2 (1).ipynb', '.ipynb_checkpoints', 'graph.png', 'timesData.csv', 'Exercices Data_Viz_plotly.ipynb']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_mal

## 1- Quels sont les pays dans lesquels le score universitaire pour le transfert de connaissances se démarque ? Utiliser un ou plusieurs graphique pour justifier votre réponse.

In [2]:
# Show country 
print(timeData.country.value_counts())

United States of America    659
United Kingdom              300
Germany                     152
Australia                   117
Canada                      108
                           ... 
Ghana                         1
Latvia                        1
Unted Kingdom                 1
Kenya                         1
Luxembourg                    1
Name: country, Length: 72, dtype: int64


In [3]:
# clean country names
df = timeData
df.country = df['country'].replace(['Unisted States of America', 'Unted Kingdom', 'Austria'],
                                   ['United States of America','United Kingdom', 'Australia'])
df.country.value_counts()

United States of America    660
United Kingdom              301
Germany                     152
Australia                   148
Canada                      108
                           ... 
Nigeria                       1
Ghana                         1
Latvia                        1
Kenya                         1
Luxembourg                    1
Name: country, Length: 69, dtype: int64

In [4]:
# check if there are null values
df.isnull().values.any()

True

In [5]:
# change the object column type to float and change the "-" to "NaN"
df.income = [float(each.replace('-', 'NaN')) for each in df.income]
df.income.dtype

dtype('float64')

In [6]:
# remove lines that contain "NaN" 
df = df.dropna()
df.income

1       83.7
2       87.5
3       64.3
5       57.0
6       73.5
        ... 
2597    83.7
2598    35.4
2599    44.0
2601    40.4
2602    39.8
Name: income, Length: 2178, dtype: float64

In [7]:
df1 = df[['country', 'income']]
#calculate the average (mean()) income per country
g = df1.groupby(['country']).mean()
g


,income
country,
Argentina,28.600000
Australia,49.615972
Belarus,28.000000
Belgium,72.536111
Brazil,44.300000
...,...
Uganda,28.000000
Ukraine,28.400000
United Arab Emirates,31.700000


In [8]:
# sort the dataframe against the income column in descending order
df1 = g.sort_values(by='income', ascending = False)
df1

,income
country,
South Africa,83.036842
Iran,74.441667
South Korea,72.924242
Belgium,72.536111
Netherlands,71.840278
...,...
Ukraine,28.400000
Latvia,28.300000
Kenya,28.000000


In [9]:
x = df1.index
y = df1.income

# import graph objects as "go"
import plotly.graph_objs as go

# Création de la trame 1
trace = go.Scatter(
                    x = x,
                    y = y,
                    mode = "lines + markers",
                    name = "score",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text = df.university_name)

layout = dict(title = 'La moyenne du score universitaire pour le revenu de l\'industrie  par pays de 2011 à 2016',
              yaxis = dict(title = 'Score universitaire pour le revenu de l\'industrie')
             )

data = [trace]
fig = dict(data = data, layout = layout)
iplot(fig)

In [10]:
import plotly.graph_objs as go

# Création de la trace 1 _ 2014
trace =go.Scatter(
                    x = x,
                    y = y,
                    mode = "markers",
                    name = "score",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = y)
layout = dict(title = 'La moyenne du score universitaire pour le revenu de l\'industrie  par pays de 2011 à 2016',
              yaxis = dict(title = 'Score universitaire pour le revenu de l\'industrie')
             )

data = [trace]
fig = dict(data = data, layout = layout)
iplot(fig)

In [11]:
trace = go.Bar(
                x = x,
                y = y,
                name = "Le score universitaire pour le transfert de connaissances par pays",
                marker = dict(color = 'rgba(255, 87, 51, 0.5)',
                             line = dict(color ='rgb(0,0,0)',width =1.5)),
                text = x)

layout = go.Layout(barmode = "group",
                  title = 'La moyenne du score universitaire pour le revenu de l\'industrie  par pays de 2011 à 2016',
                  yaxis = dict(title = 'Score universitaire pour le revenu de l\'industrie'))
fig = go.Figure(data = trace, layout = layout)
iplot(fig)

## 2- Existe-t-il une corrélation entre le score universitaire pour le transfert de connaissances et le score universitaire des perspectives internationales (personnel, étudiants, recherche) ? Utiliser un ou plusieurs graphique pour justifier votre réponse.

In [12]:
import warnings
warnings.simplefilter('ignore')

In [13]:
# change the object column type to float and change the "-" to "NaN"
df.international = [float(each.replace('-', 'NaN')) for each in df.international]


/home/ines/anaconda3/envs/MachineLearning/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
df = df.dropna()
df.international.head(30)

1      54.6
2      82.3
3      29.5
5      77.7
6      77.2
8      90.0
12     58.5
13     62.4
15     53.3
18     32.9
19     39.1
20     91.4
21     90.8
22     49.0
23     49.4
26     73.2
27     32.6
28     64.3
29     93.3
30     21.5
31     31.6
33     97.8
35     88.0
39     67.3
41     44.9
44     44.5
46     31.2
47    100.0
50     22.1
53     60.5
Name: international, dtype: float64

In [24]:
df.head(5)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
5,6,University of Cambridge,United Kingdom,90.5,77.7,94.1,94.0,57.0,91.2,"18,812",11.8,34%,46 : 54,2011
6,6,University of Oxford,United Kingdom,88.2,77.2,93.9,95.1,73.5,91.2,"19,919",11.6,34%,46 : 54,2011


In [19]:
data = df[["year","country", "income", "international"]]
#data2011 = data[['year'== 2011]]
data.head(5)
#data.info()


,year,country,income,international
1,2011,United States of America,83.7,54.6
2,2011,United States of America,87.5,82.3
3,2011,United States of America,64.3,29.5
5,2011,United Kingdom,57.0,77.7
6,2011,United Kingdom,73.5,77.2


In [20]:
df2011 = data[data.year==2011]
df2011=df2011.groupby('country').mean()

df2012 = data[data.year==2012]
df2012=df2012.groupby('country').mean()

df2013 = data[data.year==2013]
df2013=df2013.groupby('country').mean()

df2014 = data[data.year==2014]
df2014=df2014.groupby('country').mean()

df2015 = data[data.year==2015]
df2015=df2015.groupby('country').mean()

df2016 = data[data.year==2016]
df2016=df2016.groupby('country').mean()

df2=pd.concat([df2011,df2012,df2013,df2014,df2015,df2016])
df2

,year,income,international
country,,,
Australia,2011,51.387500,82.475000
Belgium,2011,97.400000,27.150000
Canada,2011,41.666667,72.266667
China,2011,63.175000,38.025000
Denmark,2011,61.033333,47.733333
...,...,...,...
Uganda,2016,28.000000,58.000000
Ukraine,2016,28.400000,38.300000
United Arab Emirates,2016,31.700000,95.300000


In [27]:
# import figure factory
import plotly.figure_factory as ff

# prepare data
df2['index']= np.arange(1, len(df2)+1) # ajout d'une colonne index pour avoir la nuance des couleurs à droite du graphe 
# scatter matrix
fig = ff.create_scatterplotmatrix(df2, diag='box',index = 'index',colormap='Portland',colormap_type='cat',height=700, width=700)
iplot(fig)

/home/ines/anaconda3/envs/MachineLearning/lib/python3.8/site-packages/plotly/tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



In [49]:
import plotly.express as px
#df = px.data.tips()
fig = px.scatter(df2, x="income", y="international", color="year", facet_col="year",
           color_continuous_scale=px.colors.sequential.Viridis)#, render_mode="webgl")
fig.show()

In [28]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(df2, x="income", y="international", color="year")
fig.show()

In [50]:
import plotly.graph_objs as go

x = df2.year

trace1 = {
  'x': x,
  'y': df2.income,
  'name': 'score income',
  'type': 'bar'
};
trace2 = {
  'x': x,
  'y': df2.international,
  'name': 'score international',
  'type': 'bar'
};
data = [trace1, trace2];
layout = {
  'xaxis': {'title': 'Top des 3 meilleurs universités'},
  'barmode': 'relative',
  'title': 'Citations et enseignement des 3 meilleures universités en 2014'
};
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [43]:
df2.index

Index(['Australia', 'Belgium', 'Canada', 'China', 'Denmark', 'Egypt',
       'Finland', 'France', 'Germany', 'Hong Kong',
       ...
       'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'Uganda',
       'Ukraine', 'United Arab Emirates', 'United Kingdom',
       'United States of America'],
      dtype='object', name='country', length=247)

In [41]:
# import graph objects as "go"
import plotly.graph_objs as go

# Création de la première trace 
trace1 = go.Bar(
                x = df2.index,
                y = df2.income,
                name = "income",
                marker = dict(color = 'rgba(255, 174, 255, 0.5)',
                             line = dict(color ='rgb(0,0,0)',width =1.5)),
                text = df2.income)

# Création de la deuxième trace 
trace2 = go.Bar(
                x = df2.index,
                y = df2.international,
                name = "international",
                marker = dict(color = 'rgba(255, 255, 128, 0.5)',
                              line = dict(color = 'rgb(0,0,0)',width = 1.5)),
                text = df2.international)
data = [trace1, trace2]
layout = go.Layout(barmode = "relative")
fig = go.Figure(data = data, layout = layout)
iplot(fig)